# Logistic Regression Consulting Project

## Solutions

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('logregconsult').getOrCreate()

In [5]:
data = spark.read.csv('/FileStore/tables/customer_churn.csv',inferSchema=True,
                     header=True)

In [6]:
data.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: string (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)
-- Churn: integer (nullable = true)

### Check out the data

In [8]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
summary| Names| Age| Total_Purchase| Account_Manager| Years| Num_Sites| Onboard_date| Location| Company| Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
 count| 900| 900| 900| 900| 900| 900| 900| 900| 900| 900|
 mean| null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777| null| null| null|0.16666666666666666|
 stddev| null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.7648355920350969| null| null| null| 0.3728852122772358|
 min| Aaron King| 22.0| 100.0| 0| 1.0| 3.0|2006-01-02 04:16:13|00103 Jeffrey Cre...| Abbott-Thompson| 0|
 max|Zachary Walsh| 65.0| 18026.01| 1| 9.15| 14.0|2016-12-28 04:07:38|Unit 9800 Box 287...|Zuniga, Clark and...| 1|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+

In [9]:
data.columns

Out[54]: ['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

### Format for MLlib

We'll ues the numerical columns. We'll include Account Manager because its easy enough, but keep in mind it probably won't be any sort of a signal because the agency mentioned its randomly assigned!

In [11]:
from pyspark.ml.feature import VectorAssembler

In [12]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [13]:
output = assembler.transform(data)

In [14]:
final_data = output.select('features','churn')

### Test Train Split

In [16]:
train_churn,test_churn = final_data.randomSplit([0.7,0.3])

### Fit the model

In [18]:
from pyspark.ml.classification import LogisticRegression

In [19]:
lr_churn = LogisticRegression(labelCol='churn', maxIter=1000)

In [20]:
fitted_churn_model = lr_churn.fit(train_churn)

In [21]:
fitted_churn_model.summary.predictions.describe().show()

+-------+------------------+-------------------+
summary| churn| prediction|
+-------+------------------+-------------------+
 count| 634| 634|
 mean|0.1608832807570978| 0.1277602523659306|
 stddev|0.3677133674974258|0.33408624185354996|
 min| 0.0| 0.0|
 max| 1.0| 1.0|
+-------+------------------+-------------------+

In [22]:
# FPR - False positive rate
# TPR - True positive rate 

fitted_churn_model.summary.roc.describe().show()

+-------+-------------------+-------------------+
summary| FPR| TPR|
+-------+-------------------+-------------------+
 count| 108| 108|
 mean|0.43925786688944585| 0.8551198257080608|
 stddev|0.31951545623684513|0.23270417072875332|
 min| 0.0| 0.0|
 max| 1.0| 1.0|
+-------+-------------------+-------------------+

### Evaluate results

Let's evaluate the results on the data set we were given (using the test data)

In [24]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [25]:
# pyspark.ml.classification.BinaryLogisticRegressionSummary OBJECT

pred_and_labels = fitted_churn_model.evaluate(test_churn)

pred_and_labels

Out[117]: <pyspark.ml.classification.BinaryLogisticRegressionSummary at 0x7f93f908ddd0>

In [26]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
 features|churn| rawPrediction| probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
[25.0,9672.03,0.0...| 0|[4.95683377222708...|[0.99301398027041...| 0.0|
[29.0,5900.78,1.0...| 0|[4.29689573170942...|[0.98657201990763...| 0.0|
[29.0,12711.15,0....| 0|[5.67364328986807...|[0.99657643450702...| 0.0|
[30.0,8874.83,0.0...| 0|[3.35361240882808...|[0.96622292982852...| 0.0|
[30.0,10183.98,1....| 0|[2.82393730623530...|[0.94395572743506...| 0.0|
[31.0,8829.83,1.0...| 0|[4.60728617063948...|[0.99011973131822...| 0.0|
[31.0,11743.24,0....| 0|[7.25652816347213...|[0.99929494410037...| 0.0|
[32.0,5756.12,0.0...| 0|[4.61563900769223...|[0.99020111056019...| 0.0|
[32.0,8011.38,0.0...| 0|[2.10102129385350...|[0.89100240353717...| 0.0|
[32.0,8617.98,1.0...| 1|[0.82643103085067...|[0.69559976165580...| 0.0|
[32.0,9036.27,0.0...| 0|[-0.4501335269593...|[0.38932901928886...| 1.0|
[32.0,10716.75,0....| 0|[4.71317573840100...|[0.99110362970698...| 0.0|
[32.0,11715.72,0....| 0|[3.53860172753246...|[0.97176637363956...| 0.0|
[32.0,12479.72,0....| 0|[4.85678857400814...|[0.99228457660538...| 0.0|
[33.0,7492.9,0.0,...| 0|[5.25168972157283...|[0.99478864148920...| 0.0|
[33.0,11370.28,1....| 0|[7.05064445967769...|[0.99913390065102...| 0.0|
[33.0,12115.91,1....| 0|[2.27458417016368...|[0.90675012264821...| 0.0|
[34.0,9270.9,0.0,...| 0|[5.38197036577346...|[0.99542229993223...| 0.0|
[34.0,9779.12,0.0...| 0|[2.84857653976918...|[0.94524505565652...| 0.0|
[34.0,12397.49,0....| 0|[3.24576551074309...|[0.96252065378433...| 0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 20 rows

In [27]:
test_churn.groupBy('Churn').count().show()

+-----+-----+
Churn|count|
+-----+-----+
 1| 48|
 0| 218|
+-----+-----+

In [28]:
# Matriz de Confusão

pred_and_labels.predictions.groupBy('churn','prediction').count().show()

+-----+----------+-----+
churn|prediction|count|
+-----+----------+-----+
 1| 0.0| 21|
 0| 0.0| 206|
 1| 1.0| 27|
 0| 1.0| 12|
+-----+----------+-----+

In [29]:
print('Accuracy: %f' % pred_and_labels.accuracy)
print('AUC:      %f' % pred_and_labels.areaUnderROC)
print('weightedRecall:      %f' % pred_and_labels.weightedRecall)
print('weightedPrecision:      %f' % pred_and_labels.weightedPrecision)

Accuracy: 0.875940
AUC: 0.869075
weightedRecall: 0.875940
weightedPrecision: 0.868659

### Using AUC

In [31]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='churn')

In [32]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [33]:
auc

Out[124]: 0.7537270642201834

[Common question - what is a good AUC value?](https://stats.stackexchange.com/questions/113326/what-is-a-good-auc-for-a-precision-recall-curve)

### Predict on brand new unlabeled data

We still need to evaluate the new_customers.csv file!

In [36]:
final_lr_model = lr_churn.fit(final_data)

In [37]:
new_customers = spark.read.csv('/FileStore/tables/new_customers.csv',inferSchema=True,
                              header=True)

In [38]:
new_customers.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: string (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)

In [39]:
test_new_customers = assembler.transform(new_customers)

In [40]:
test_new_customers.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: string (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)
-- features: vector (nullable = true)

In [41]:
final_results = final_lr_model.transform(test_new_customers)

In [42]:
final_results.select('Names','Company','prediction').show()

+--------------+----------------+----------+
 Names| Company|prediction|
+--------------+----------------+----------+
 Andrew Mccall| King Ltd| 0.0|
Michele Wright| Cannon-Benson| 1.0|
 Jeremy Chang|Barron-Robertson| 1.0|
Megan Ferguson| Sexton-Golden| 1.0|
 Taylor Young| Wood LLC| 0.0|
 Jessica Drake| Parks-Robbins| 1.0|
+--------------+----------------+----------+

Ok! That is it! Now we know that we should assign Acocunt Managers to Cannon-Benson,Barron-Robertson,Sexton-GOlden, and Parks-Robbins!

## Great Job!